In [1]:
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense


In [2]:
# download dataset from GCP to keras if not yet downloaded

cache_path = pathlib.Path("../root/.keras/datasets/data")
if not cache_path.is_dir():
  dataset_url = "https://storage.googleapis.com/derma-data/raw_data/archive.zip"
  data_dir = tf.keras.utils.get_file(
                                    origin=dataset_url,
                                      extract=True,
                                      archive_format	='zip')
  data_dir = pathlib.Path(data_dir)

9766127229/9766127229 [==============================] - 269s 0us/step


In [3]:
batch_size = 32
img_height = 150
img_width = 150

In [4]:
test_ds = tf.keras.utils.image_dataset_from_directory('../root/.keras/datasets/data/test'
                                        ,batch_size=batch_size
                                        ,label_mode='categorical'
                                        ,image_size=(img_height,img_width))

Found 2606 files belonging to 8 classes.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory('../root/.keras/datasets/data/valid'
                                        ,batch_size=batch_size
                                        ,label_mode='categorical'
                                        ,image_size=(img_height,img_width))

Found 2546 files belonging to 8 classes.


In [16]:
for x, y in val_ds.take(1):
  print(x.shape)

(32, 150, 150, 3)


In [17]:
# Load model
from keras.applications.densenet import DenseNet201

def load_model():

  model = DenseNet201(
      include_top=False,
      weights="imagenet",
      input_shape=x[0].shape,
      classifier_activation="softmax")
        
  return model


In [1]:
# Number of parameters
model = load_model()
model.summary()

In [18]:
# deactivate the training of the DenseNet parameters
def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
    
    return model

In [2]:
# check params of EfficientNet are non-trainable
model = set_nontrainable_layers(model)
model.summary()

In [19]:
#Chain pre-trained layers of EfficienNet with our flattening and dense layers
from tensorflow.keras import layers, models

class_names = test_ds.class_names
num_classes = len(class_names)

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer1 = layers.Dense(512, activation='relu')
    dense_layer2 = layers.Dense(128, activation='relu')
    dense_layer3 = layers.Dense(64, activation='relu')
    dense_layer4 = layers.Dense(32, activation='relu')
    prediction_layer = layers.Dense(num_classes, activation='softmax')
        
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer1,
        dense_layer2,
        dense_layer3,
        dense_layer4,
        prediction_layer
    ])
    
    return model

In [3]:
# Number of parameters for our customized EfficientNet
model = add_last_layers(model)
model.summary()

In [28]:
# Build a full customized DenseNet and compile it
from tensorflow.keras import optimizers

def build_model():
    
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=[
tf.keras.metrics.BinaryAccuracy(name='accuracy'),
tf.keras.metrics.Precision(name='precision'),
tf.keras.metrics.Recall(name='recall')
])
    return model

In [29]:
# Pass the validation and test datasets into the DenseNet preprocessing function -required-
from tensorflow import keras
from keras.applications.densenet import preprocess_input

def new_preproc(X,y):
  return preprocess_input(X),y

val_preproc_ds = val_ds.map(new_preproc)
test_preproc_ds = test_ds.map(new_preproc)

In [30]:
# Create a train_generator using keras ImageDataGenerator and flow_from_directory functions
# Notice that the required DenseNet preprocess_input is passed onto the train dataset directly
# in the parameters of the Image Data Generator function

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range = 10, zoom_range = 0.20, 
fill_mode = "nearest", shear_range = 0.1, horizontal_flip = True, 
width_shift_range = 0.1, height_shift_range = 0.1)

train_generator = train_datagen.flow_from_directory(directory = '../root/.keras/datasets/data/train',
target_size=(img_height,img_width), classes= test_ds.class_names,
class_mode='categorical', batch_size=batch_size, shuffle=True)

Found 20179 images belonging to 8 classes.


In [31]:
# Fit the model
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

csv_logger = CSVLogger('training.log')

# checkpoint_callback = ModelCheckpoint(
#                 filepath=LOCAL_CHECKPOINT_PATH,
#                 save_weights_only=True,
#                 monitor='val_accuracy',
#                 mode='max',
#                 save_best_only=True)

history = model.fit(train_generator, 
                    validation_data=val_preproc_ds,
                    epochs=30,
                    batch_size=batch_size,
                    callbacks=[es, csv_logger]
                    )


Epoch 1/30
631/631 [==============================] - 595s 922ms/step - loss: 1.1159 - accuracy: 0.9168 - precision: 0.7453 - recall: 0.5080 - val_loss: 0.9726 - val_accuracy: 0.9205 - val_precision: 0.7851 - val_recall: 0.5008
Epoch 2/30
631/631 [==============================] - 576s 912ms/step - loss: 0.9274 - accuracy: 0.9249 - precision: 0.7748 - recall: 0.5631 - val_loss: 0.9184 - val_accuracy: 0.9271 - val_precision: 0.8238 - val_recall: 0.5306
Epoch 3/30
631/631 [==============================] - 572s 906ms/step - loss: 0.8731 - accuracy: 0.9288 - precision: 0.7841 - recall: 0.5935 - val_loss: 0.8770 - val_accuracy: 0.9274 - val_precision: 0.7911 - val_recall: 0.5695
Epoch 4/30
631/631 [==============================] - 575s 911ms/step - loss: 0.8457 - accuracy: 0.9296 - precision: 0.7818 - recall: 0.6059 - val_loss: 0.8520 - val_accuracy: 0.9288 - val_precision: 0.7790 - val_recall: 0.6009
Epoch 5/30
631/631 [==============================] - 575s 911ms/step - loss: 0.8130 - a

In [32]:
# Plot the accuracy
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [4]:
plot_history(history)

In [33]:
# Evaluate the model
res_vgg = model.evaluate(test_preproc_ds)

test_accuracy_vgg = res_vgg[1]
test_precision_vgg = res_vgg[2]
test_recall_vgg = res_vgg[3]

print(f"test_accuracy_vgg = {round(test_accuracy_vgg,2)*100} %")
print(f"test_precision_vgg = {round(test_precision_vgg,2)*100} %")
print(f"test_recall_vgg = {round(test_recall_vgg,2)*100} %")

82/82 [==============================] - 29s 340ms/step - loss: 0.8863 - accuracy: 0.9325 - precision: 0.7583 - recall: 0.6754
test_accuracy_vgg = 68.0 %


In [34]:
res_vgg


[0.8863046765327454, 0.9325115084648132, 0.758293867111206, 0.6753645539283752]